# Reverse engineer molecules

#### Apr 2024 version

## Enumerate signatures and molecules from Morgan fingerprints

In [1]:
# CHANGE TO YOUR DIRECTORY PATH
path_directory = "C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/github/signature"

# packages
import numpy as np
import os

import pandas as pd
import sys
import time

np.set_printoptions(threshold=sys.maxsize)

os.chdir(path_directory)

from rdkit.Chem.Descriptors import ExactMolWt
from rdkit import Chem
from rdkit import RDLogger

RDLogger.DisableLog("rdApp.*")

# Reverse engineer molecules from Morgan
from src.signature.enumerate_signature import enumerate_molecule_from_signature, enumerate_signature_from_morgan
from src.signature.enumerate_utils import reduced_fingerprint
from src.signature.signature import signature_neighbor
from src.signature.signature_alphabet import (
    load_alphabet,
    morgan_vector_from_signature,
    SignatureAlphabet,
    signature_from_smiles,
)
from src.signature.utils import read_csv

In [2]:
# paths
file_smiles = "./datasets/MetaNetX_weight_500_radius_3_size_1000"
file_alphabet = "./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048"

# Load Smiles file
H, D = read_csv(file_smiles)
print(H, D.shape[0])
Smiles = np.asarray(list(set(D[:, 0])))
print(f"Number of smiles: {len(Smiles)}")

# Get alphabet
Alphabet = load_alphabet(file_alphabet)
Alphabet.print_out()

['SMILES', 'SIG', 'SIG-NBIT'] 1000
Number of smiles: 1000
filename: ./datasets/MetaNetX_weight_500_Alphabet_radius_3_nBits_2048.npz
radius: 3
nBits: 2048
splitcomponent: False
isomericSmiles: False
formalCharge: True
atomMapping: False
kekuleSmiles: False
allHsExplicit: False
maxvalence: 4
alphabet length: 551908


In [3]:
# fonctions test
def test_sol_ECFP(smis):
    fpgen = AllChem.GetMorganGenerator(radius=Alphabet.radius, fpSize=2048)
    ecfp_list = []
    for smi in smis:
        mol = Chem.MolFromSmiles(smi)
        morgan = fpgen.GetCountFingerprint(mol).ToList()
        ecfp_list.append(morgan)
    return len(set(tuple(i) for i in ecfp_list)) == 1

def test_sol_ECFP_reduced(smis):
    ecfp_list = []
    for smi in smis:
        morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False).tolist()
        ecfp_list.append(morgan)
    return len(set(tuple(i) for i in ecfp_list)) == 1

def test_sol_sig(smis):
    sig_list = []
    for smi in smis:
        sig = signature_from_smiles(smi, Alphabet, neighbor=True)[0]
        sign = signature_neighbor(sig)
        sig_list.append(sign)
    return len(set(tuple(i) for i in sig_list)) == 1

- ECFP reduced => Signature

In [4]:
list_i = []
list_smi = []
list_nsig = []
list_foundsig = []
list_ct = []
list_ct_solve = []
list_timeout = []

In [5]:
Alphabet.nBits = 2048
max_nbr_partition = int(1e5)

for i in range(5):
    smi = D[i, 0]
    print(i, "|", smi)
    sig, mol, smi = signature_from_smiles(smi, Alphabet, neighbor=True, verbose=False)
    sign = signature_neighbor(sig)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)
    st = time.time()

    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sign in Ssig
    ft = time.time() - st

    print(f"{i} | {smi} | {len(Ssig)} | {int(foundsig)} | {ft:.4f} | {ct_solve:.4f}| {bool_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_nsig.append(len(Ssig))
    list_foundsig.append(foundsig)
    list_ct.append(ft)
    list_ct_solve.append(ct_solve)
    list_timeout.append(bool_timeout)

0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O | 1 | 1 | 24.5737 | 0.0094| False
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O | 1 | 1 | 22.1008 | 0.0050| False
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O | 1 | 1 | 36.8614 | 0.0081| False
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C | 1 | 1 | 47.9009 | 0.0150| False
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N | 1 | 1 | 21.9072 | 0.0030| False


In [6]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "Nsig": list_nsig,
        "FoundSig": list_foundsig,
        "CT enum ecfp_red to sig": list_ct,
        "CT solve by partitions": list_ct_solve,
        "Partitions timeout": list_timeout,
    }
)

df

# df.to_excel("RevSig1_7_ecfp_to_sig", index=False)

ID                                                smi  Nsig  FoundSig  \
0   0            CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O     1      True   
1   1          OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O     1      True   
2   2               O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O     1      True   
3   3  CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...     1      True   
4   4                     COC(=O)c1sc(=S)n(-c2ccccc2)c1N     1      True   

   CT enum ecfp_red to sig  CT solve by partitions  Partitions timeout  
0                24.573678                0.009373               False  
1                22.100818                0.004997               False  
2                36.861432                0.008092               False  
3                47.900865                0.015023               False  
4                21.907235                0.003000               False

- Signature => Smiles

In [7]:
list_i = []
list_smi = []
list_wt = []
list_nmol = []
list_foundmol = []
list_ct_mol = []
list_recursion_timeout = []

In [8]:
Alphabet.nBits = 0
max_nbr_recursion = 1e3
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nmol | FoundMol | CPU-time | Timeout")
for i in range(10):
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))
    sig, mol, smi = signature_from_smiles(smi, Alphabet, neighbor=True)
    sign = signature_neighbor(sig)
    st = time.time()

    smol, recursion_timeout = enumerate_molecule_from_signature(
        sign,
        Alphabet,
        smi,
        max_nbr_recursion=max_nbr_recursion,
        max_nbr_solution=max_nbr_solution,
        repeat=repeat,
        verbose=False,
    )

    ft = time.time() - st
    foundmol = smi in smol
    if foundmol == False:
        print(smol)

    print(f"{i} | {smi} | {wt:.1f} | {len(smol)} | {int(foundmol)} | {ft:.4f} | {recursion_timeout}")

    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nmol.append(len(smol))
    list_foundmol.append(int(foundmol))
    list_ct_mol.append(ft)
    list_recursion_timeout.append(recursion_timeout)

ID | smi | weigth | Nmol | FoundMol | CPU-time | Timeout
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O | 286.1 | 1 | 1 | 0.0610 | False
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O | 346.0 | 1 | 1 | 0.0920 | False
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O | 322.1 | 1 | 1 | 0.6957 | True
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C | 420.3 | 2 | 1 | 0.5361 | False
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N
4 | COC(=O)c1sc(=S)n(-c2ccccc2)c1N | 266.0 | 1 | 1 | 0.0860 | False
5 | CCC(O)C(O)C(CCl)C(Cl)C(=O)C(=O)O
5 | CCC(O)C(O)C(CCl)C(Cl)C(=O)C(=O)O | 272.0 | 1 | 1 | 0.0470 | False
6 | CC1C(=O)c2ccc3c(c2C(c2ccc4c(c2)OCO4)C1C)OCO3
6 | CC1C(=O)c2ccc3c(c2C(c2ccc4c(c2)OCO4)C1C)OCO3 | 338.1 | 3 | 1 | 0.9270 | True
7 | COc1ccc(O)cc1-c1oc2c(OC)c(OC)cc(O)c2c(=O)c1OC
7 | COc1ccc(O)cc1-c1oc2c(OC)c(OC)cc(O)

In [9]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "weight": list_wt,
        "Nmol": list_nmol,
        "FoundMol": list_foundmol,
        "CT enum sig to mol": list_ct_mol,
        "Recursion timeout": list_recursion_timeout,
    }
)

df

# df.to_excel("RevSig_rad3_sig_to_mol", index=False)

ID                                                smi      weight  Nmol  \
0   0            CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O  286.116486     1   
1   1          OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O  345.969705     1   
2   2               O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O  322.131742     1   
3   3  CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...  420.272318     2   
4   4                     COC(=O)c1sc(=S)n(-c2ccccc2)c1N  266.018370     1   
5   5                   CCC(O)C(O)C(CCl)C(Cl)C(=O)C(=O)O  272.021829     1   
6   6       CC1C(=O)c2ccc3c(c2C(c2ccc4c(c2)OCO4)C1C)OCO3  338.115424     3   
7   7      COc1ccc(O)cc1-c1oc2c(OC)c(OC)cc(O)c2c(=O)c1OC  374.100168     1   
8   8                  CCCCC=CCC(O)C(O)C(O)C(O)CCCC(=O)O  304.188589     1   
9   9     Cc1c(CCC(=O)N2CCCC2)c(=O)n2c3ccccc3n(C)c2c1C#N  362.174276     1   

   FoundMol  CT enum sig to mol  Recursion timeout  
0         1            0.061007              False  
1         1            0.092001              False  
2         1            0.695663               True  
3         1            0.536086              False  
4         1            0.086019              False  
5         1            0.047000              False  
6         1            0.927012               True  
7         1            0.370746              False  
8         1            0.088706              False  
9         1            0.142283              False

- ECFP reduced => Signature => Smiles

In [10]:
list_i = []
list_smi = []
list_wt = []
list_nsig = []
list_nsigtrue = []
list_foundsig = []
list_nmol = []
list_foundmol = []
list_ct_sig = []
list_ct_mol = []
list_ct_all = []
list_ct_solve = []

In [11]:
max_nbr_partition = int(1e5)
max_nbr_recursion = 1e5
max_nbr_solution = 1001
repeat = 10

print(f"ID | smi | weigth | Nsig | NsigTrue | FoundSig | Nmol | FoundMol | CPU-time")
for i in range(5):
    # preparation
    smi = D[i, 0]
    print(i, "|", smi)
    wt = ExactMolWt(Chem.MolFromSmiles(smi))
    sig, mol, smi = signature_from_smiles(smi, Alphabet, neighbor=True, verbose=False)
    sign = signature_neighbor(sig)
    morgan = reduced_fingerprint(smi, radius=Alphabet.radius, useFeatures=False)

    # ecfp_red => signature(s)
    st_1 = time.time()
    Alphabet.nBits = 2048
    Ssig, bool_timeout, ct_solve = enumerate_signature_from_morgan(
        morgan, Alphabet, max_nbr_partition=max_nbr_partition, method="partitions", verbose=False
    )
    foundsig = sign in Ssig
    ct_sig = time.time() - st_1
    print(f"...enumeratesignature: nbr signatures {len(Ssig)} CPU-time {ct_sig:.4f}")

    # signature(s) => molecule(s)
    st_2 = time.time()
    Smol, Nsig = set(), 0
    Alphabet.nBits = 0
    for j in range(len(Ssig)):
        # print("Ssig[j]", Ssig[j])
        smol, recursion_timeout = enumerate_molecule_from_signature(
            Ssig[j],
            Alphabet,
            smi,
            max_nbr_recursion=max_nbr_recursion,
            max_nbr_solution=max_nbr_solution,
            repeat=repeat,
            verbose=False,
        )
        if len(smol):
            Nsig += 1
            print(f"...enumeratemolecule:  signature {j} nbr molecules {len(smol)}")
        Smol = Smol | set(smol)
    ct_mol = time.time() - st_2

    ct_total = time.time() - st_1
                
    Smolfinal = []
    for smi2 in Smol:
        if test_sol_ECFP_reduced([smi, smi2]):
            Smolfinal.append(smi2)
    if len(Smol) != len(Smolfinal):
        print("Smol", len(Smol), "Smolfinal", len(Smolfinal))

    foundmol = smi in Smolfinal
    if foundmol == False:
        print(Smolfinal)
        
    print(
        f"{i} | {smi} | {wt:.1f} | {len(Ssig)} | {Nsig} | {int(foundsig)} | {len(Smolfinal)} | {int(foundmol)} | {ct_total:.4f} | {ct_solve:.4f}"
    )
    
    list_i.append(i)
    list_smi.append(smi)
    list_wt.append(wt)
    list_nsig.append(len(Ssig))
    list_nsigtrue.append(Nsig)
    list_foundsig.append(int(foundsig))
    list_nmol.append(len(Smolfinal))
    list_foundmol.append(int(foundmol))
    list_ct_sig.append(ct_sig)
    list_ct_mol.append(ct_mol)
    list_ct_all.append(ct_total)
    list_ct_solve.append(ct_solve)

ID | smi | weigth | Nsig | NsigTrue | FoundSig | Nmol | FoundMol | CPU-time
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O
...enumeratesignature: nbr signatures 1 CPU-time 23.9325
...enumeratemolecule:  signature 0 nbr molecules 1
0 | CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O | 286.1 | 1 | 1 | 1 | 1 | 1 | 23.9945 | 0.0060
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O
...enumeratesignature: nbr signatures 1 CPU-time 21.9720
...enumeratemolecule:  signature 0 nbr molecules 1
1 | OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O | 346.0 | 1 | 1 | 1 | 1 | 1 | 22.0659 | 0.0061
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O
...enumeratesignature: nbr signatures 1 CPU-time 36.1901
...enumeratemolecule:  signature 0 nbr molecules 1
2 | O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O | 322.1 | 1 | 1 | 1 | 1 | 1 | 36.7489 | 0.0080
3 | CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)(O)C(O)C1C
...enumeratesignature: nbr signatures 1 CPU-time 48.3493
...enumeratemolecule:  signature 0 nbr molecules 2
Smol 2 Smolfinal 1
3 | CCC1O

In [12]:
df = pd.DataFrame(
    {
        "ID": list_i,
        "smi": list_smi,
        "wt": list_wt,
        "Nsig": list_nsig,
        "NsigTrue": list_nsigtrue,
        "FoundSig": list_foundsig,
        "Nmol": list_nmol,
        "Foundmol": list_foundmol,
        "CT ecfp_sig": list_ct_sig,
        "CT sig_mol": list_ct_mol,
        "CT ecfp_mol": list_ct_all,
        "CT solve_partitions": list_ct_solve,
    }
)

df

# df.to_excel("RevSig_rad3_ecfp_to_mol.xlsx", index=False)

ID                                                smi          wt  Nsig  \
0   0            CCN(C(C)O)C(O)C=NC=C(C)C(=O)C(=O)C(=O)O  286.116486     1   
1   1          OC1C(O)C2CC1C1C3(Cl)CC(Cl)(C(Cl)C3Cl)C21O  345.969705     1   
2   2               O=C(O)CCc1nc2ccccc2n(CCc2ccccc2)c1=O  322.131742     1   
3   3  CCC1OC(=O)C(C)C(O)C(C)C(O)C(C)(O)CC(C)C(O)C(C)...  420.272318     1   
4   4                     COC(=O)c1sc(=S)n(-c2ccccc2)c1N  266.018370     1   

   NsigTrue  FoundSig  Nmol  Foundmol  CT ecfp_sig  CT sig_mol  CT ecfp_mol  \
0         1         1     1         1    23.932479    0.061015    23.994479   
1         1         1     1         1    21.971973    0.092899    22.065883   
2         1         1     1         1    36.190052    0.557832    36.748886   
3         1         1     1         1    48.349327    0.397936    48.747263   
4         1         1     1         1    21.850166    0.089259    21.940386   

   CT solve_partitions  
0             0.006007  
1             0.006107  
2             0.007999  
3             0.017098  
4             0.003010